In [1]:
!pip install catboost

     |████████████████████████████████| 66.2MB 44kB/s 


In [2]:
import numpy as np
import pandas as pd

In [3]:
from catboost import CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=100, 
    task_type = 'GPU',
    logging_level = 'Silent'
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=0)

In [4]:
from sklearn.metrics import accuracy_score
from itertools import combinations

In [5]:
def cv_cat(train):
  result = []
  X = train[train.columns.difference(['y'])]
  y = train.loc[:, 'y']
  for k, (train_index,test_index) in enumerate(kf.split(data)):
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
    cat_fit = cb_model.fit(X_train, y_train, cat_features=list(train.select_dtypes(include=['object']).columns))
    pred = cat_fit.predict(X_valid)
    true = y_valid.to_numpy().flatten()
    error = accuracy_score(true, pred)
    result.append(error)
  return (np.mean(result))

In [6]:
from sklearn.datasets import make_classification

In [7]:
X,y = make_classification(n_samples=1000, n_features=30, n_informative=10,n_redundant=2 ,n_repeated=0, n_classes=2, class_sep=1.0, hypercube=True,scale=1.0, shuffle=False, random_state=None)

In [8]:
data = pd.DataFrame(X)
data.columns = ['x'+str(i) for i in range(data.shape[1])]
data['y'] = y

In [ ]:
feature_list = list(set(data.columns.tolist())-set('y'))
used_feature = []
unused_feature = feature_list
num_feature = 1
direction = "up"

while num_feature <= data.shape[1]:
  if (direction=="up"):
    var_comb1 = [t+tuple(used_feature) for t in list(combinations(unused_feature, 1))]
  else:
    var_comb1 = list(combinations(used_features,num_feature-1))
  if (num_feature>1):
    var_comb2 = list((used_feature,))
  else: 
    var_comb2 = [] 
  variable_df = pd.DataFrame()
  variable_df['variable'] = var_comb1+var_comb2
  variable_df['ACC'] = 0

  for k in range(variable_df.shape[0]):
    using_var = list(variable_df.iloc[k,0]+('y',))
    variable_df.iloc[k,1] = cv_cat(data.loc[:, using_var])
    print(using_var,k,variable_df.iloc[k,1])

  idx = [int(variable_df['ACC'].idxmax()),]
  min_variable = variable_df.iloc[idx].iloc[0,0]

  if (len(min_variable)==num_feature):
    num_feature = num_feature+1
    direction = "up"
  else:
    num_feature = num_feature-1
    direction = "down"
  used_feature = min_variable
  unused_feature = list(set(feature_list)-set(used_feature))
  print(num_feature, used_feature)

['x18', 'y'] 0 0.4939790089490688
['x25', 'y'] 1 0.49699100297902693
['x13', 'y'] 2 0.5370250490010969
['x16', 'y'] 3 0.48201794608980236
['x24', 'y'] 4 0.48600996205786623
['x9', 'y'] 5 0.5950111788435142
['x15', 'y'] 6 0.502004999010987
['x7', 'y'] 7 0.5119910329491167
['x22', 'y'] 8 0.499991008973045
['x5', 'y'] 9 0.687990385595176
['x11', 'y'] 10 0.5870061678444912
['x10', 'y'] 11 0.534040028052004
['x23', 'y'] 12 0.5649871428314541
['x26', 'y'] 13 0.4990229750708792
['x19', 'y'] 14 0.4849939760119401
['x29', 'y'] 15 0.4910179640718562
['x12', 'y'] 16 0.4990079900259541
['x4', 'y'] 17 0.4880179580778383
['x28', 'y'] 18 0.4790029550508592
['x3', 'y'] 19 0.5170290050529571
['x2', 'y'] 20 0.5579981178783574
['x8', 'y'] 21 0.6150042257826689
['x0', 'y'] 22 0.7299694904485324
['x1', 'y'] 23 0.5580220939502376
['x21', 'y'] 24 0.5160729591867317
['x6', 'y'] 25 0.47604490718263176
['x14', 'y'] 26 0.5169840499181816
['x17', 'y'] 27 0.46899893905881934
['x20', 'y'] 28 0.5259720798642954
['x2

In [ ]:
variable_df